# 文書分類とその入力

>- 文書分類：「文書」を何らかのカテゴリーに分類するタスクのこと
>- スパムメールであるか否かの判別、評判分析

In [1]:
import janome
from janome.tokenizer import Tokenizer

t = Tokenizer()  # 形態素解析器をtとする
for token in t.tokenize('私は秋田犬が大好き。'):  # 解析結果を1単語ずつtokenに保存していく 
    print(token)

私	名詞,代名詞,一般,*,*,*,私,ワタシ,ワタシ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
秋田	名詞,固有名詞,地域,一般,*,*,秋田,アキタ,アキタ
犬	名詞,一般,*,*,*,*,犬,イヌ,イヌ
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
大好き	名詞,形容動詞語幹,*,*,*,*,大好き,ダイスキ,ダイスキ
。	記号,句点,*,*,*,*,。,。,。


In [2]:
for token in t.tokenize('私は秋田犬が大好き。', wakati=True):  # wakati=Trueで単語分割を目的とした分かち書きを行う
    print(token, end='/')  # end='/'で、単語ごとに「/」で区切る指定を加える

私/は/秋田/犬/が/大好き/。/

In [3]:
words = [token for token in t.tokenize('私は秋田犬が大好き。', wakati=True)]  # 単語分割の結果をリスト型にして取っておく

# N-gram

In [4]:
words  #uni-gram

['私', 'は', '秋田', '犬', 'が', '大好き', '。']

In [5]:
words[0:2]

['私', 'は']

In [6]:
words[1:3]

['は', '秋田']

In [7]:
words[2:4]

['秋田', '犬']

In [8]:
# N-gramを得るには、左側のインデックスにnを足した数を右側のインデックスに指定する必要がある
# bi-gramなら、[1:3],[2:4]など、左の数字に2を足すと右の数字になる

def get_word_n_grams(sentence, n):  # 引数をsentence(解析を行う文章)とn(N-gramのnの値)とするget_word_n_grams関数
    words = [token for token in t.tokenize(sentence, wakati=True)]  # 単語分割の結果をリスト型にして取ったもの
    result = []  # 結果を保存するリスト
    for index in range(len(words)):  # indexには0,1,2,...の順番に値が入る
        result.append(words[index: index+n])
        if index + n >= len(words):  # 指定しなければN-gramを算出できない状況が発生
            return result

In [9]:
sentence = '私は秋田犬が大好き。'
get_word_n_grams(sentence, 1)  # 引数が1→uni-gram

[['私'], ['は'], ['秋田'], ['犬'], ['が'], ['大好き'], ['。']]

In [10]:
get_word_n_grams(sentence, 2)  #bi-gram

[['私', 'は'], ['は', '秋田'], ['秋田', '犬'], ['犬', 'が'], ['が', '大好き'], ['大好き', '。']]

In [11]:
get_word_n_grams(sentence, 3)  #tri-gram

[['私', 'は', '秋田'],
 ['は', '秋田', '犬'],
 ['秋田', '犬', 'が'],
 ['犬', 'が', '大好き'],
 ['が', '大好き', '。']]

In [12]:
# 文字のN-gram
def get_character_n_grams(sentence, n):
    result = []
    for index in range(len(sentence)):  # 文字単位→文章を直接入れるだけ
        result.append(sentence[index: index+n])
        if index + n >= len(sentence):
            return result

In [13]:
get_character_n_grams(sentence, 1)

['私', 'は', '秋', '田', '犬', 'が', '大', '好', 'き', '。']

In [14]:
get_character_n_grams(sentence, 2)

['私は', 'は秋', '秋田', '田犬', '犬が', 'が大', '大好', '好き', 'き。']

In [15]:
get_character_n_grams(sentence, 3)

['私は秋', 'は秋田', '秋田犬', '田犬が', '犬が大', 'が大好', '大好き', '好き。']

# Bag-of-words

>- 単語の出現する順番は考えない
>- 「私は秋田犬が大好き。」と「秋田犬は私が大好き。」は同じ表現となる
>- 「犬に関する文書か否か」を分類するだけならBOWで分類可能

In [16]:
sentence2 = '私は犬が少し苦手。'
get_word_n_grams(sentence2, 1)

[['私'], ['は'], ['犬'], ['が'], ['少し'], ['苦手'], ['。']]

In [17]:
sentence3 = '秋田犬は私が大好き。'
get_word_n_grams(sentence3, 1)

[['秋田'], ['犬'], ['は'], ['私'], ['が'], ['大好き'], ['。']]

In [18]:
def get_bag_of_words(document):  # 文書document中の単語の出現回数を数える関数
    result_dict = {}  # 辞書
    words=[token for token in t.tokenize(document, wakati=True)]  # 単語分割の結果リスト
    for word in words:
        if word not in result_dict:  # 語wordが辞書になければ
          result_dict[word]=1        # 出現回数を1とする
        else:                        # 辞書にあれば
          result_dict[word] += 1     # これまでの出現回数に１を加える
    return result_dict

In [19]:
document1 = '私は秋田犬が大好き。秋田犬は私が大好き。'  # 2つの文章は、BOW上では同じ表現である
document2 = '私は犬が少し苦手。'

In [20]:
get_bag_of_words(document1)

{'私': 2, 'は': 2, '秋田': 2, '犬': 2, 'が': 2, '大好き': 2, '。': 2}

In [21]:
get_bag_of_words(document2)

{'私': 1, 'は': 1, '犬': 1, 'が': 1, '少し': 1, '苦手': 1, '。': 1}

>- 用例ベクトル：文書や文一つ一つの用例を、それぞれ一つのベクトルとして表したもの
>-  ベクトル空間モデル：様々な言語的アイテムをベクトルとして表す数理モデルのこと
>- ある単語をある数値と紐づけて表現する→名義尺度
>- 単語や品詞など、カテゴリが決まっているだけのデータを変数にする→ダミー変数

>- BOWにおいては、「用例中にある単語が出てくる出現回数」をベクトルとする
>- 「私」をx1、「は」をx2、...などとし、コーパス中の語彙サイズをNとするとxNまでの変数が必要となる
>- 素性：x1,x2等の変数、素性値：その変数の値

In [22]:
def make_dictionary(documents):  # 素性の辞書を作成する関数
    result_dict = {}
    index=1
    for adocument in documents:
      words=[token for token in t.tokenize(adocument, wakati=True)]
      for word in words:
        if word not in result_dict:
          result_dict[word]=index
          index+=1
    return result_dict

In [23]:
t = Tokenizer()
document1 = '私は秋田犬が大好き。秋田犬は私が大好き。'
document2 = '私は犬が少し苦手。'
documents = [document1, document2]
dict = make_dictionary(documents)
dict

{'私': 1, 'は': 2, '秋田': 3, '犬': 4, 'が': 5, '大好き': 6, '。': 7, '少し': 8, '苦手': 9}

>- 次元の高い(語彙サイズの大きい)文書の場合、用例ベクトルがほとんど0となる→スパース(疎)なベクトル

In [24]:
def make_BOW_vectors(documents, dictionary):  # 用例ベクトルを作成する関数、辞書にある単語の出現回数
  result_vectors=[]
  for adocument in documents:
    avector={}
    words=[token for token in t.tokenize(adocument, wakati=True)]
    for entry in dictionary:
      avector[dictionary[entry]]=0
    for word in words:
      avector[dictionary[word]]+=1
    result_vectors.append(avector)
  return result_vectors

In [25]:
make_BOW_vectors(documents, dict)  # 1~9はmake_dictionary関数で作成した素性の辞書における単語に対応している

[{1: 2, 2: 2, 3: 2, 4: 2, 5: 2, 6: 2, 7: 2, 8: 0, 9: 0},
 {1: 1, 2: 1, 3: 0, 4: 1, 5: 1, 6: 0, 7: 1, 8: 1, 9: 1}]

>- 「uni-gramのBag-of-wordsの出現」を素性とする方法→コーパスサイズを考慮したN-gramが実現
>- 対応する単語が出現すれば1、出現しなければ0と素性値を指定する

In [26]:
def make_BOW_onehot_vectors(documents, dictionary):  #「素性の辞書にあれば1、なければ0を返す」
  result_vectors=[]
  for adocument in documents:
    avector={}
    words=[token for token in t.tokenize(adocument, wakati=True)]
    for entry in dictionary:
      avector[dictionary[entry]]=0
    for word in words:
      avector[dictionary[word]]=1  # ここがさっきと異なる
    result_vectors.append(avector)
  return result_vectors

In [27]:
make_BOW_onehot_vectors(documents, dict)

[{1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 0, 9: 0},
 {1: 1, 2: 1, 3: 0, 4: 1, 5: 1, 6: 0, 7: 1, 8: 1, 9: 1}]

>- CountVectorizerモジュールによる単語の出現頻度の文書ベクトル
>- CountVectorizer{sklearn}では、コーパスからBOWの出現回数の行列に変換が可能
>- 用例数×素性数の行列

In [28]:
def make_corpus(documents):  # 文書からコーパスを作成する関数
  result_corpus=[]
  for adocument in documents:
    words=[token for token in t.tokenize(adocument, wakati=True)]
    text=" ".join(words)  # join関数によって、文字列を結合する
    result_corpus.append(text)
  return result_corpus

In [29]:
t = Tokenizer()
document1 = '私は秋田犬が大好き。秋田犬は私が大好き。'
document2 = '私は犬が少し苦手。'
documents = [document1, document2]
corpus = make_corpus(documents)
corpus

['私 は 秋田 犬 が 大好き 。 秋田 犬 は 私 が 大好き 。', '私 は 犬 が 少し 苦手 。']

In [30]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern='(?u)\\b\\w+\\b', ngram_range=(1,2))  
#一文字の単語を許容するように、token_patternを指定する
# ngram_rangeのベクトルで、nの値を指定(今回はuni-gramとbi-gramの出現回数の行列を表示するように指定)

X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names_out(corpus))  # dictに対応
print(X.toarray())  # 疎な行列Xを0が見える形の行列形式に変換

['が' 'が 大好き' 'が 少し' 'は' 'は 犬' 'は 私' 'は 秋田' '大好き' '大好き 秋田' '少し' '少し 苦手' '犬'
 '犬 が' '犬 は' '私' '私 が' '私 は' '秋田' '秋田 犬' '苦手']
[[2 2 0 2 0 1 1 2 1 0 0 2 1 1 2 1 1 2 2 0]
 [1 0 1 1 1 0 0 0 0 1 1 1 1 0 1 0 1 0 0 1]]


# TF-IDF

* TF-IDF：単語の重要度を考慮した値
* 参考：https://dev.classmethod.jp/articles/yoshim_2017ad_tfidf_1-2/
* 文書を分類するために重要な単語と重要でない単語が存在する
* たとえば、音楽に関する文書を分類する際に、楽器の名前は重要だが、食べ物の名前は重要でない
* 重要な単語は、その文書内における出現頻度が高い、かつ、他の文書内にはあまり出現しない
* TF：単語の出現頻度(Term Frequency)、IDF：単語の逆文書頻度(Inverse Document Frequency)

* $ 単語の出現回数：tf_{i,j} = n_{i,j} \\ 単語の出現比率：tf_{i,j} = \frac{n_{i,j}}{\sum_k{n_{i,j}}} \\ 逆文書頻度：idf_{i,j} = \log{\frac{|D|}{|d:w_i \in d|}} \\ d：単語w_iを含む文書数 \\ |D|：コーパス内の総文書数 $

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(token_pattern='(?u)\\b\\w+\\b', ngram_range=(1,2))
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names_out(corpus))
print(X.toarray())

['が' 'が 大好き' 'が 少し' 'は' 'は 犬' 'は 私' 'は 秋田' '大好き' '大好き 秋田' '少し' '少し 苦手' '犬'
 '犬 が' '犬 は' '私' '私 が' '私 は' '秋田' '秋田 犬' '苦手']
[[0.25932077 0.36446629 0.         0.25932077 0.         0.18223315
  0.18223315 0.36446629 0.18223315 0.         0.         0.25932077
  0.12966039 0.18223315 0.25932077 0.18223315 0.12966039 0.36446629
  0.36446629 0.        ]
 [0.25096919 0.         0.35272845 0.25096919 0.35272845 0.
  0.         0.         0.         0.35272845 0.35272845 0.25096919
  0.25096919 0.         0.25096919 0.         0.25096919 0.
  0.         0.35272845]]


# LSA(Latent Semantic Analysis)

>- LSA(潜在意味解析)：疎な行列である用例ベクトルの行列を、低次元の意味空間に落とし込むことで、密な行列にするという手法
>- 主成分分析に近い手法
>- 語彙サイズの考慮、類義語・多義語の考慮
>- 特異値分解によって、素性数×文書数の行列 = 単語数×トピック数の行列 + トピック数分の特異値 + トピック数×文書数の行列とする
>- 特異値の順番で、上位r個のトピックを選び、任意の要素に次元削減可能

In [32]:
document1='私は秋田犬が大好き。秋田犬は私が大好き。'
document2='私は犬が少し苦手。'
document3='私はぶたとペンギンが好きです。ペンギンは鳥の仲間ですが、水族館で見ることができます。'
document4='隣の家の犬はよく吠える犬です。'
document5='ゴロピカドンが好きです。'
document6='事務室のカギが見つからないと思ったが、よく探したらあった。'
document7='フルーツはどれも大好きですが、私の好物はイチゴです。'
document8='りんごもミカンもパイナップルも、どれもそれぞれとても美味しい果物です。'
document9='私はフルーツが好きで、よくフルーツパーラーに食べに行きます。'
document10='ぶどうとメロンは必需品。'
documents=[document1, document2,document3, document4, document5, document6, document7, document8, document9, document10]

corpus=make_corpus(documents)
print(corpus)

['私 は 秋田 犬 が 大好き 。 秋田 犬 は 私 が 大好き 。', '私 は 犬 が 少し 苦手 。', '私 は ぶた と ペンギン が 好き です 。 ペンギン は 鳥 の 仲間 です が 、 水族館 で 見る こと が でき ます 。', '隣 の 家 の 犬 は よく 吠える 犬 です 。', 'ゴロピカドン が 好き です 。', '事務 室 の カギ が 見つから ない と 思っ た が 、 よく 探し たら あっ た 。', 'フルーツ は どれ も 大好き です が 、 私 の 好物 は イチゴ です 。', 'りんご も ミカン も パイナップル も 、 どれ も それぞれ とても 美味しい 果物 です 。', '私 は フルーツ が 好き で 、 よく フルーツパーラー に 食べ に 行き ます 。', 'ぶどう と メロン は 必需 品 。']


In [33]:
vectorizer = TfidfVectorizer(token_pattern='(?u)\\b\\w+\\b', sublinear_tf=True, use_idf=True)
 # sublinear_tf=False, use_idf=Trueがデフォルト　use_idf=Falseにすると、どの単語もIDF=1となる
X = vectorizer.fit_transform(corpus)
print(X.shape) #文書やrandom_stateのシードの数によって結果が異なります

(10, 57)


In [34]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=7, n_iter=5, random_state=42)  
  # ncomponents=7：上位7つの主要な要素に次元削減、n_iter=5：5回繰り返す、random_state=42：set.seed(42)と同じ
newX = svd.fit_transform(X)
print(newX) #文書やrandom_stateのシードの数によって結果が異なります

[[ 0.63766684 -0.41904035  0.23741743 -0.10413639 -0.08510271  0.03668821
  -0.247584  ]
 [ 0.56856989 -0.4931894   0.15201046 -0.1299718  -0.07666711 -0.01388097
  -0.27937207]
 [ 0.58417091  0.22669084 -0.37712668 -0.0455379  -0.12673344 -0.13179484
   0.08388958]
 [ 0.51624345 -0.25649865  0.13492393  0.1127026   0.34340273 -0.23962655
   0.66353769]
 [ 0.42976803  0.42018574 -0.31602309 -0.27390951  0.02095586 -0.52088956
  -0.19645644]
 [ 0.25962181 -0.00758344 -0.35962843  0.48400399  0.66850129  0.17891578
  -0.29967936]
 [ 0.67412904  0.28524     0.34269431  0.06485862 -0.03318881  0.08831955
  -0.0092058 ]
 [ 0.21590475  0.61211124  0.58765645  0.21852602  0.02763261  0.12820898
  -0.05333523]
 [ 0.45732415  0.13258818 -0.39092101 -0.26445099 -0.15173526  0.63941905
   0.21545951]
 [ 0.17729441 -0.09072649 -0.20928661  0.73785634 -0.5761692  -0.09609122
   0.03714835]]


In [35]:
print(newX.shape)  # 語彙サイズ57のうち、50は疎なベクトル

(10, 7)


In [36]:
# 寄与率
svd.explained_variance_ratio_ #文書やrandom_stateのシードの数によって結果が異なります

array([0.03667877, 0.15012475, 0.14335443, 0.12162776, 0.12126485,
       0.10472207, 0.09752666])

In [37]:
# 累積寄与率
svd.explained_variance_ratio_.sum()  
# 次元数と累積寄与率はトレードオフ、次元数を増やせば累積寄与率は高くなる、調整が必要

0.77529930095747